In [2]:
import numpy as np

In [1]:
import os

In [1]:
!pip install -q tensorflow-text

In [14]:
!pip install pandas 

     |████████████████████████████████| 9.7 MB 6.1 MB/s eta 0:00:01
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)


In [4]:
!pip install requests

  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached urllib3-1.26.4-py2.py3-none-any.whl (153 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)


In [2]:
!pip install -q tf-models-official

In [342]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer
import pandas as pd 
import matplotlib.pyplot as plt
from pathlib import Path    
tf.get_logger().setLevel('ERROR')
AUTOTUNE = tf.data.AUTOTUNE

In [343]:
img_height, img_width = 244, 244
image_count = 8500

In [344]:
text_dataset = tf.data.TextLineDataset("data/train.jsonl")
text_dataset = text_dataset.shuffle(image_count, reshuffle_each_iteration=False)

In [345]:
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # resize the image to the desired size
    return tf.image.resize(img, [img_height, img_width])

In [346]:
img_path = 'data/img/'

def process_path(text_l):
    text_l = tf.strings.regex_replace(text_l, '"', '')


    parts = tf.strings.split(text_l, ":", maxsplit=-1)
    
    
    label = parts[3]
    label = tf.strings.split(label, ",")[0]
    label = tf.strings.to_number(label, tf.int32) 

    text = parts[-1]
    
    file_path = parts[1]
    file_path = tf.strings.split(file_path, ",")[0]

    file_path = tf.cond(tf.strings.length(file_path) == tf.constant(4), 
                        lambda: tf.add("0", file_path), lambda: file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(img_path+file_path+".png")
    img = decode_img(img)
    return {"img_input": img, "text_input": text}, {"classifier": label}

In [347]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
text_dataset = text_dataset.map(process_path, num_parallel_calls=AUTOTUNE)
# val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)


In [348]:
val_size = int(image_count * 0.2)
train_ds = text_dataset.skip(val_size)
val_ds = text_dataset.take(val_size)
# print(tf.data.experimental.cardinality(train_ds).numpy())
# print(tf.data.experimental.cardinality(val_ds).numpy())

In [352]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size=32)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds



In [353]:
train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

In [360]:
for image, label in val_ds.take(1):
    print("Image shape: ", image)
    print("Label: ", label)


Image shape:  {'img_input': <tf.Tensor: shape=(32, 244, 244, 3), dtype=float32, numpy=
array([[[[1.13030037e+02, 1.13030037e+02, 1.13030037e+02],
         [1.12262299e+02, 1.12262299e+02, 1.12262299e+02],
         [1.12680618e+02, 1.12680618e+02, 1.12680618e+02],
         ...,
         [4.63945694e+01, 4.63945694e+01, 4.63945694e+01],
         [4.36751709e+01, 4.36751709e+01, 4.36751709e+01],
         [4.96196327e+01, 4.96196327e+01, 4.96196327e+01]],

        [[2.12174255e+02, 2.12174255e+02, 2.12174255e+02],
         [2.10568558e+02, 2.10568558e+02, 2.10568558e+02],
         [2.10094254e+02, 2.10094254e+02, 2.10094254e+02],
         ...,
         [9.46646042e+01, 9.46646042e+01, 9.46646042e+01],
         [9.21288834e+01, 9.21288834e+01, 9.21288834e+01],
         [8.28103561e+01, 8.28103561e+01, 8.28103561e+01]],

        [[2.10860657e+02, 2.10860657e+02, 2.10860657e+02],
         [2.10420929e+02, 2.10420929e+02, 2.10420929e+02],
         [2.12398163e+02, 2.12398163e+02, 2.12398163e+0